# Classification non supervisée

**TP noté, à déposer sur Claroline (OMI_5AP_SMCG) en fin de séance **

Ce TP s'intéresse à deux méthodes de classification non supervisée: les $k$-means et les modèles de mélanges gaussiens.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GMM
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap

## $k$-means

#### Génération des données

Pour commmencer, on génère des variables aléatoires sur $\mathbb{R}^2$ selon la loi:
$$ p \mathcal{N}(c_1,Id) + (1-p)\mathcal{N}(c_2,Id),$$
où $c_1$ et $c_2$ sont deux points de $\mathbb{R}^2$.

In [ ]:
n_samples=750
p=0.65
centers=np.array([[ -1.60834549, 1.3667341], [1.40887718, -1.5253229]])


X=np.random.normal(size=2*n_samples).reshape(2,n_samples)


delta=np.random.binomial(1,p,size=n_samples).reshape(1,-1)
X=(np.dot(centers[0].reshape(-1,1),delta)+ np.dot(centers[1].reshape(-1,1),1-delta))+X
X=X.transpose()

plt.scatter(X[:,0],X[:,1],c=delta,cmap="spring")
plt.title("Melange de gaussiennes");

#### Résultats

On applique la méthode `KMeans` classer les points.

In [ ]:
y_pred = KMeans(n_clusters=2).fit_predict(X)


plt.scatter(X[:,0],X[:,1],c=y_pred,cmap="spring")
plt.title("Melange de gaussiennes")
plt.show()

### Cas insatisfaisants

#### Mauvais nombre de clusters
Si le nombre $k$ de clusters est mal choisi, on obtient alors une résultat qui ne correspond pas à ce que l'on souhaite.

In [ ]:
y_pred = KMeans(n_clusters=3).fit_predict(X)

xmin=min(X.ravel())
xmax=max(X.ravel())
plt.xlim(xmin,xmax)  # set the xlim to xmin, xmax
plt.ylim(xmin,xmax)
plt.scatter(X[:,0],X[:,1],c=y_pred,cmap="spring")
plt.title("Melange de gaussiennes: mauvais nombre de clusters")
plt.show()

#### Anisotropie
Autre cas où les $k$-means se révèlent peu efficaces: si les gaussiennes qui génèrent les variables aléatoires sont anisotropes, les centroides ne séparent pas *correctement* les données.

In [ ]:
sigma=[[2,4],[4,18]]
p=0.65
centers=np.array([[ -1.60834549, 4.63667341], [1.40887718, -2.85253229]])


X=np.random.normal(size=2*n_samples).reshape(2,n_samples)
L=np.linalg.cholesky(sigma)
X=np.dot(L,X)

delta=np.random.binomial(1,p,size=n_samples).reshape(1,-1)
X=(np.dot(centers[0].reshape(-1,1),delta)+ np.dot(centers[1].reshape(-1,1),1-delta))+X
X=X.transpose()

y_pred = KMeans(n_clusters=2).fit_predict(X)

plt.scatter(X[:,0],X[:,1],c=y_pred,cmap="spring")
plt.show()

#### Support non linéaire

Voici encore un cas pour lequel les $k$-means seuls ne fonctionnent pas.
Si le support de la mesure de probabilité qui génère les variables aléatoires n'est pas convexe, cela peut aussi poser problème.

In [ ]:
n_samples = 1500
np.random.seed(0)
p = 0.7
Zc= np.random.rand(1, n_samples)<p
Z = (np.random.rand(1, n_samples)-1+Zc)
t = np.pi * Z
x = np.sin(t)
y = (t-3*np.pi/4)*t*(t+3*np.pi/4)

X = np.concatenate((x, y))
X = X.T
plt.figure(figsize=(40, 20))
plt.subplot(243)
plt.scatter(X[:, 0], X[:, 1], c=Zc, cmap="spring");

y_pred = KMeans(n_clusters=2).fit_predict(X)
plt.subplot(242)
plt.scatter(X[:, 0], X[:, 1], c=y_pred, cmap="spring");


#### ISOMAP

Pour parer à ce dernier problème, on peut utiliser un algorithme de réduction de dimension.
Isomap est un algorithme qui permet de définir une nouvelle distance sur les données, qui rend compte de la structure des données.
La classe `Isomap` de `scikitlearn.manifold` implémente cette algorithme.
Une matrice contenant les distances est renvoyée.

Elle est ici contenue dans `distmat`.
*La distance entre `X[i]` et `X[j]` est dans `distmat[i][j]` ou `distmat[j][i]`.*

In [ ]:
man=Isomap(n_neighbors=int(np.ceil(np.log(n_samples)))).fit(X)
distmat=man.dist_matrix_

#### ISOMAP et $k$-means

La distance étant définie uniquement sur les points observés, on définit le minimum sur les centroides sur les points observés.

On va utiliser la matrice `distmat` que l'on a généré avec ISOMAP pour construire les $k$-means.
Pour ça, on va implémenter pas à pas l'algorithme de LLoyd.

*Exercice :*
**Définir une fonction `distorsion` qui prend deux arguments: une liste d'indices `list` et un indice `i`; et renvoie **
$$ \sum_{j\in list} d(X[j],X[i])^2 $$

In [ ]:
def distorsion(X,index):
    

**À l'aide de la fonction `distorsion`, définir une fonction `centroides` qui prend deux arguments: les données `X` et une liste de deux indices `c_index` et renvoie les centroides (i.e. barycentres) de chaque cellule de Voronoï définie par les centres`X[c_index[0]]`.et `X[c_index[1]]`.**

In [ ]:
def better_centroid(X,c_index):

**Définir une fonction `lloyd` qui prend en arguments les données `X`, et deux paramètres `init` et `epsilon` et execute l'algorithme de Lloyd avec un nombre `init` d'initilisations aléatoires et s'arrête à chaque fois si les nouveaux centroides n'améliorent pas la distorsion de plus de `epsilon`.
La fonction renvoie alors le centroides et la distorsion.**

In [ ]:
def lloyd(X,init,epsilon):

**Tester votre code:**

In [ ]:
dis,c=lloyd(X,10,10**-6)
print dis,c
predict=np.array([np.argmin([distmat[i][c[0]],distmat[i][c[1]]]) for i in range(n_samples)])

plt.scatter(X[:, 0], X[:, 1],c=predict, cmap="spring");

## Modèles de mélange gaussien

La deuxième méthode que l'on va étudier est celle de l'estimateur du maximum de vraisemblance des mélanges gaussiens.

### Cas homoscédastique

#### Génération des données
Dans un premier temps, on s'intéresse au cas où les données sont généré par un mélange de deux gaussiennes ayant la même matrice de covariance.

On commence par générer ces données.

In [ ]:
n_samples=750

sigma=[[2,4],[4,18]]
p=0.65
centers=np.array([[ -1.60834549, 4.63667341], [1.40887718, -2.85253229]])


X=np.random.normal(size=2*n_samples).reshape(2,n_samples)
L=np.linalg.cholesky(sigma)
X=np.dot(L,X)

delta=np.random.binomial(1,p,size=n_samples).reshape(1,-1)
X=(np.dot(centers[0].reshape(-1,1),delta)+ np.dot(centers[1].reshape(-1,1),1-delta))+X
X=X.transpose()

xmin=min(X.ravel())
xmax=max(X.ravel())
plt.xlim(xmin,xmax)
plt.ylim(xmin,xmax)
plt.scatter(X[:,0],X[:,1],c=delta,cmap="spring");

#### Résultats

*Exercice: *
**Utiliser la méthode GMM dans le cas hétéroscédastique, sans contrainte sur la matrice de covariance**

In [ ]:
help(GMM)

### Cas hétéroscédastique - cas unidimensionnel
#### Génération des données
*Exercice: *
**Modifier le code précédent pour simuler des données suivant la loi 
$$ p\mathcal{N}(m_1,\sigma_1) + (1-p)\mathcal{N}(m_2,\sigma_2) $$**

In [ ]:
n_samples=750

sigma1=[[2,4],[4,18]]
sigma2=[[10,4],[4,2]]
p=0.65
centers=np.array([[ -1.60834549, 4.63667341], [1.40887718, -2.85253229]])


#
#
#

xmin=min(X.ravel())
xmax=max(X.ravel())
plt.xlim(xmin,xmax)
plt.ylim(xmin,xmax)
plt.scatter(X[:,0],X[:,1],c=delta,cmap="spring");

#### Résultats

*Exercice:*
**Implémenter l'algorithme EM pour la dimension 1, avec deux composantes et sur la contrainte $\sigma_1/\sigma_2>C$ et $\sigma_2/\sigma_1>C$:**

    initialiser EM aléatoirement
    appliquer l'algorithme EM
    arrêter si sigma_2/sigma_1<C ou sigma_1/sigma_2<C
    recommencer
    

In [ ]:


xmin=min(X.ravel())
xmax=max(X.ravel())
plt.xlim(xmin,xmax)
plt.ylim(xmin,xmax)
plt.scatter(X[0,:],X[1,:],c=y)